# Test Data of Melanie: Load and compare to 1000G SNPs

In [1]:
import allel
import h5py  # Python Package to do the HDF5.
import numpy as np
import pandas as pd
import socket
import os as os
import sys as sys
import multiprocessing as mp

socket_name = socket.gethostname()
print(socket_name)
if socket_name == "VioletQueen":
    path = "/home/harald/git/HAPSBURG/"   # The Path on Harald's machine
elif socket_name.startswith("midway2"):
    print("Midway jnovmbre partition detected.")
    path = "/project2/jnovembre/hringbauer/HAPSBURG/"  # The Path on Midway Cluster
else: 
    raise RuntimeWarning("Not compatible machine. Check!!")
    
os.chdir(path)  # Set the right Path (in line with Atom default)

print(os.getcwd()) # Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns
print(f"CPU Count: {mp.cpu_count()}")

sys.path.insert(0,"./package/")  # hack to get local package first in path
from hapsburg.PackagesSupport.loadEigenstrat.loadEigenstrat import load_eigenstrat

midway2-0402.rcc.local
Midway jnovmbre partition detected.
/project2/jnovembre/hringbauer/HAPSBURG
CPU Count: 28


### Load the Data of Melanie and

In [6]:
%time
path_es = "./Data/Melanie/ROH"
es = load_eigenstrat(base_path=path_es)

Eigenstrat packed: False
3 Eigenstrat Files with 2 Individuals and 5480936 SNPs


In [7]:
es.load_ind_df()

,iid,sex,cls
0,CLR44,U,ROH
1,COL336,U,ROH


In [53]:
%%time
df_snp = es.load_snp_df()

CPU times: user 40.1 s, sys: 1.25 s, total: 41.4 s
Wall time: 41.4 s


### Load the data of Davids 1240K

In [10]:
%%time
path_es1 = "./Data/ReichLabEigenstrat/Raw.v42.4/v42.4.1240K"
es1 = load_eigenstrat(base_path=path_es1)
df_snp1 = es1.load_snp_df()

Eigenstrat packed: True
3 Eigenstrat Files with 6676 Individuals and 1233013 SNPs


In [19]:
df_snp2 = df_snp1[df_snp1["chr"]==4]
print(f"Exctracted {len(df_snp2)} SNPs on Chromosome 4")

Exctracted 71634 SNPs on Chromosome 4


### Compare the two different 1240K SNP frames

In [17]:
df = pd.merge(df_snp, df_snp2, on="pos")

In [20]:
idx = (df["ref_x"] == df["ref_y"])
idx2 = (df["alt_x"] == df["alt_y"])
print(f"Matching Ref Allele: {np.sum(idx)}")
print(f"Matching Alt Allele: {np.sum(idx2)}")

Matching Ref Allele: 68759
Matching Alt Allele: 68720


In [21]:
len(df)

68759

# Check how many SNPs are covered in Melanie's file [stand alone]

In [27]:
%%time
iid = "COL336"  # CLR44
path_es = "./Data/Melanie/ROH"

################################
es = load_eigenstrat(base_path=path_es)
df_ind = es.load_ind_df()
df_snp = es.load_snp_df()

Eigenstrat packed: False
3 Eigenstrat Files with 2 Individuals and 5480936 SNPs
CPU times: user 1min 6s, sys: 2.13 s, total: 1min 8s
Wall time: 1min 8s


In [40]:
%%time
gt2 = es.get_geno_indvidiaul_iid("CLR44")

CPU times: user 22.4 s, sys: 172 ms, total: 22.6 s
Wall time: 22.6 s


In [45]:
np.sum(gt2<3)/len(gt2)

0.1999609190838937

In [39]:
pd.value_counts(gt1)  # COL336

3    4375573
2    1067938
0      37425
dtype: int64

In [44]:
np.sum(gt1<3)/len(gt1)

0.20167413011208304

In [ ]:
df_snp

In [48]:
df_save = df_snp[["chr","pos"]][gt1<3]

In [50]:
len(df_save)

1105363

In [ ]:
def save_1240kmarkers():
    """Save all 1240 Markers in csv"""
    df_snp = pd.read_csv(snp1240k_path, header=None, sep=r"\s+", engine="python")
    df_snp.columns = ["SNP", "chr", "map", "pos", "ref", "alt"]
    df_snp = df_snp[df_snp["chr"] == ch]
    print(f"Loaded {len(df_snp)} Chr.{ch} SNPs.")

    df_save = df_snp[["chr", "pos"]]
    df_save.to_csv(marker_path, sep="\t", header=None, index=False)
    print(f"Saved {len(df_save)} 1240k Markers on Chr. {ch} to {marker_path}")